In [ ]:
from pathlib import Path

import torch

from ssl_brainmet.models.build_nnunet_model import build_model_from_plans

In [2]:
weights_path = "/home/vincent/repos/ssl-bm/weights/cnn3d_nnunet_firstgood.pt"

In [3]:
project_dir = Path(".").resolve().parents[0]

model = build_model_from_plans(
    project_dir / "ssl_brainmet/config/nnUNetPlans.json",
    project_dir / "ssl_brainmet/config/dataset.json",
    configuration="3d_fullres",
    deep_supervision=True,
)

In [ ]:
state_dict = torch.load(weights_path)

In [ ]:
# Filter the state dict to only keep weights for the encoder
encoder_state_dict = {k[len("encoder."):]: v for k, v in state_dict.items() if k.startswith("encoder.")}

# Load the filtered weights into the encoder
model.encoder.load_state_dict(encoder_state_dict)

In [6]:
torch.save(
    {'network_weights': model.state_dict()},
    "/home/valentin/data/target/data/pretrained_weights/pretrained_weights.pt")

In [ ]:
encoder_params = {k: v for k, v in model.encoder.state_dict().items()}
decoder_encoder_params = {k: v for k, v in model.decoder.encoder.state_dict().items()}

for key in encoder_params:
    assert torch.equal(encoder_params[key], decoder_encoder_params[key]), f"Mismatch in {key}"
print("Encoder weights are consistent.")


In [ ]:
encoder_params['stages.2.0.convs.0.all_modules.1.weight']

In [ ]:
decoder_encoder_params['stages.2.0.convs.0.all_modules.1.weight']